# PyG を用いた GNN
https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html#common-benchmark-datasets

In [1]:
import torch
from torch_geometric.data import Data

## グラフデータの作成
節点 $V = {0, 1, 2}$、枝 $E = {(0,1), (1,2)}$ の無向グラフを作成。それぞれ特徴量 $x$ をもつ。  
有向グラフをもとにした記述に注意。

In [2]:
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)

枝の記述が気に食わないなら、contiguous を使うとよい。

In [3]:
edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index.t().contiguous())

data の内容。データの要素名と tensor の形がわかる

In [4]:
data

Data(x=[3, 1], edge_index=[2, 4])

グラフ構造データの要素名について

In [5]:
print(data.keys)

print('\"edge_attr\" found in data ? : {}'.format('edge_attr' in data))

['x', 'edge_index']
"edge_attr" found in data ? : False


data の要素を取り出す

In [6]:
print(data['x'])

tensor([[-1.],
        [ 0.],
        [ 1.]])


その他グラフを解析するメソッド

In [7]:
print(data.num_nodes)
print(data.num_edges)
print(data.num_node_features)
print(data.has_isolated_nodes())
print(data.has_self_loops())
print(data.is_directed())

3
4
1
False
False
False


In [8]:
data

Data(x=[3, 1], edge_index=[2, 4])

## データセット
ベンチマークの（基礎的な）データが多く用意されている。  
データセット "ENZYMES" のダウンロード

In [9]:
from torch_geometric.datasets import TUDataset

In [10]:
dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')

In [11]:
print(len(dataset))
print(dataset.num_classes)
print(dataset.num_node_features)

600
6
3


データセットには多くのグラフデータがおさめられている

In [12]:
print(dataset[0])
print(data.is_undirected())

Data(edge_index=[2, 168], x=[37, 3], y=[1])
True


上の例だと、一番目のグラフデータはノード数 37、枝数 168/2 の無向グラフで、各ノードが３次元の特徴量をもつ

データを操作するメソッド

In [13]:
print('スライス（サンプリング）')
print(dataset[:540])
print(dataset[540:])

print('\nシャッフル')
print(dataset.shuffle()[0])

スライス（サンプリング）
ENZYMES(540)
ENZYMES(60)

シャッフル
Data(edge_index=[2, 18], x=[5, 3], y=[1])


データセット "Cora" のダウンロード  
論文の引用・被引用のネットワーク

In [14]:
from torch_geometric.datasets import Planetoid

In [15]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')

In [29]:
print(len(dataset))
print(dataset[0])
print(dataset[0].num_nodes)
print(dataset[0].is_undirected())
print(dataset.num_classes)
print(dataset.num_node_features)

1
Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
2708
True
7
1433


Cora は1つだけのグラフデータからなるデータセット。このグラフはノード数 2708 の、論文間の引用関係を表す無向グラフである。各ノードは論文を表し、7つの分野にクラス分けされている。また各ノードは 1433 単語の出現の有無を 0/1 で記したベクトルを、特徴量としてもつ。

In [17]:
data = dataset[0]
print(data)

print(data.is_undirected())

print(data.train_mask.sum().item())

print(data.val_mask.sum().item())

print(data.test_mask.sum().item())

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
True
140
500
1000


- train_mask はどのノードに対してトレーニングするか決める。今回は140個のノード  
- val_mask はどのノードでバリデーション（テストデータによる能力検証）するか決める？今回は500個のノード   
- test_mask はどのノードでテストするか決める。

ミニバッチ学習（データをサンプリングして1エポックの学習を行う）のためのデータは torch_geometric.loader.DataLoader でつくる

In [18]:
from torch_geometric.loader import DataLoader

In [19]:
dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

for batch in loader:
    print(batch)
    print(batch.num_graphs)
    print('')

DataBatch(edge_index=[2, 3828], x=[1023, 21], y=[32], batch=[1023], ptr=[33])
32

DataBatch(edge_index=[2, 3870], x=[1002, 21], y=[32], batch=[1002], ptr=[33])
32

DataBatch(edge_index=[2, 4056], x=[1045, 21], y=[32], batch=[1045], ptr=[33])
32

DataBatch(edge_index=[2, 4222], x=[1083, 21], y=[32], batch=[1083], ptr=[33])
32

DataBatch(edge_index=[2, 4326], x=[1135, 21], y=[32], batch=[1135], ptr=[33])
32

DataBatch(edge_index=[2, 4302], x=[1137, 21], y=[32], batch=[1137], ptr=[33])
32

DataBatch(edge_index=[2, 4322], x=[1158, 21], y=[32], batch=[1158], ptr=[33])
32

DataBatch(edge_index=[2, 3848], x=[1003, 21], y=[32], batch=[1003], ptr=[33])
32

DataBatch(edge_index=[2, 4118], x=[1062, 21], y=[32], batch=[1062], ptr=[33])
32

DataBatch(edge_index=[2, 3840], x=[971, 21], y=[32], batch=[971], ptr=[33])
32

DataBatch(edge_index=[2, 3884], x=[1033, 21], y=[32], batch=[1033], ptr=[33])
32

DataBatch(edge_index=[2, 3834], x=[1047, 21], y=[32], batch=[1047], ptr=[33])
32

DataBatch(edge_ind

## データトランスフォーム

画像や3Dなどのデータを変換する機能がある。 
詳細は書かないが、点群を最近傍グラフにするなど出来る。

## 学習法とグラフ

単純なGCN層を使用し、Coraの論文引用データセットで実験を再現する。  
http://tkipf.github.io/graph-convolutional-networks/

In [20]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')

print(dataset)

Cora()


２層GCNを構築する。

In [21]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

In [22]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # 第1層 各特徴量ベクトル 1433 -> 16
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        # 第2層 各特徴量ベクトル 16 -> 7
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

pytorch はクラス torch.nn.Module を継承してモデルを作成。  
順伝播メソッド forward を定義して準備完了。  

super() : クラス定義中に親クラス（torch.nn.Module）のメソッドを呼び出す。self の親戚みたいなもの  
super().\_\_init\_\_() : 親クラスの初期化子を利用 

第1層 GCN ReLU ドロップアウト  
第2層 GCN softmax

トレーニングノードについて200エポックの学習を行う。

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

学習したモデルを評価

In [24]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.8040


In [32]:
dataset[0]

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [28]:
dataset[0].train_mask

tensor([ True,  True,  True,  ..., False, False, False])

In [36]:
model(data).argmax(dim=1)

tensor([3, 4, 4,  ..., 0, 3, 3])